In [1]:
import pandas as pd
import numpy as np
import numpy.matlib
import string
import math 
import collections
from numpy import linalg as LA
import random

In [2]:
COLONY_SIZE = 100

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dictionary = pd.read_csv('/content/drive/My Drive/IS_HW2/dictionary.csv').to_numpy()
coded_words = pd.read_csv('/content/drive/My Drive/IS_HW2/coded_words.csv').to_numpy()

In [5]:
english_alphabet = np.array(list(str(string.ascii_lowercase)))

In [6]:
colony=np.empty( (COLONY_SIZE,26), dtype = object)
for person in range(COLONY_SIZE):
    colony[ person] = english_alphabet
    np.random.shuffle( colony[ person] )
for person in range(COLONY_SIZE ):
    np.random.shuffle( colony[ person ] )

In [7]:
def key_dict(key):
    dictionary = {}
    base = np.array(list(str(string.ascii_lowercase)))
    for i in range(26):
        dictionary[ key[i] ] = base[i]
    return dictionary

In [8]:
def change_string(in_string, key_dict):
    array_string = np.array( list( in_string ) )
    temp_string = np.empty( len(list( in_string )), dtype=object )
    for string_letter in range(array_string.shape[0]):
        temp_string[string_letter] = key_dict[ array_string[ string_letter ] ]
    return ''.join(temp_string)

In [9]:
def rank(key, coded_words, dictionary):
    key_ = key_dict(key)
    matched = 0
    size = coded_words.shape[0]
    for word in coded_words:
        if change_string(word[0], key_) in dictionary:
            matched += 1
    return matched/size

In [10]:
def selection(population, colony_size, coded_words, dictionary):
    selected = np.empty( (colony_size,26), dtype = object)
    ranking = np.zeros(population.shape[0])
    for person in range(population.shape[0]):
        ranking[person] = rank(population[person], coded_words, dictionary)
    max_arg = (-ranking).argsort()[:colony_size]
    most_matched = ranking[max_arg[0]]
    for i in range(colony_size):
        selected[ i ] = population[ max_arg[ i ] ]
    return most_matched, selected

In [11]:
def create(parent, indice):
    english_alphabet = np.array(list(str(string.ascii_lowercase)))
    childeren = np.empty((2,26), dtype = object)
    childeren[:, 0: indice] = parent[:, 0: indice]
    childeren[0,  indice:26] = parent[1,  indice:26]
    childeren[1, indice:26] = parent[0,  indice:26]
    temp=[[],[]]
    temp1=[[],[]]
    
    for i in range(2):
        for letter in childeren[i,  indice:26]:
            if letter not in parent[i,  indice:26] :
                temp[i].append(letter)
    for i in range(2):
        for j in range(26-indice):
            if childeren[i,  indice+j] not in parent[i,  indice:26] :
                childeren[i, indice + j ] = temp[1-i][0]
                temp[1-i].remove(temp[1-i][0])
    return childeren


In [12]:
def generate(population):
    childeren = np.empty((population.shape[0],26), dtype = object)
    colapse_indices = np.random.randint(2,25, size=int(population.shape[0]/2))
    for i in range(int(population.shape[0]/2)):
        childeren[2*i:2*i+2, :] = create(population[2*i:2*i+2, :], colapse_indices[i])
    return childeren

In [13]:
def cross_over(population):
    colony_size = population.shape[0]
    best_half = population[ int(colony_size/2) : colony_size , : ]
    worst_half = population[ 0 : int(colony_size/2) , : ]
    random_index = np.arange(int(colony_size/2))
    np.random.shuffle(random_index)
    childeren_1 = generate(np.take(best_half, random_index[0:int(colony_size*(0.4))], axis = 0 ))
    childeren_2 = generate(np.take(worst_half, random_index[0:int(colony_size*(0.4))], axis = 0 ))
    selected = np.empty( (int(colony_size*(0.2)),26), dtype = object)
    selected[0:int(colony_size*(0.1))] = np.take(best_half, random_index[int(colony_size*(0.4)):int(colony_size*(0.5))], axis = 0 )
    selected[int(colony_size*(0.1)) : int(colony_size*(0.2))] = np.take(worst_half, random_index[int(colony_size*(0.4)) : int(colony_size*(0.5))], axis = 0 )
    np.random.shuffle(selected)
    childeren_3 = generate(selected)
    all_species = np.empty( (colony_size*(2),26), dtype = object)
    all_species[0:colony_size] = population
    all_species[colony_size:int(colony_size*(1.4))] = childeren_1
    all_species[int(colony_size*(1.4)) : int(colony_size*(1.8))] = childeren_2
    all_species[int(colony_size*(1.8)) : int(colony_size*(2))] = childeren_3
    return all_species

In [14]:
def colapse(childeren, indices):
    p_1 = [[],[]]
    for i in range(26):
        if indices[i] == 0:
            p_1[0].append(childeren[i])
        elif indices[i] == 1:
            p_1[1].append(childeren[i])
    random.shuffle(p_1[0])
    immortals = np.empty((1,26),dtype=object)
    for i in range(26):
        if indices[i] == 0:
            immortals[0,i]= p_1[0][0]
            p_1[0].remove(p_1[0][0])
        elif indices[i] == 1:
            immortals[0,i]= p_1[1][0]
            p_1[1].remove(p_1[1][0])
    return immortals
            

In [15]:
def mutation(population, rate):
    if np.floor(population.shape[0]*rate) > 0 :
        random_index = np.arange(population.shape[0])
        np.random.shuffle(random_index)
    immortals = np.take(population, random_index[0:int(np.floor(population.shape[0]*rate))],axis=0)
    a=np.zeros(13, dtype = int)
    b=np.ones(13, dtype = int)
    temp = np.concatenate((a, b), axis=None)
    colapse_indices = np.zeros((immortals.shape[0],26),dtype=int)
    for i in range(immortals.shape[0]):
        colapse_indices[i,:] = temp
        numpy.random.shuffle( colapse_indices[ i , : ] )
    avengers = np.empty((immortals.shape[0],26),dtype=object)
    for i in range(immortals.shape[0]):
        avengers[i,:]=colapse(immortals[i,:],colapse_indices[i,:])
    return avengers

In [16]:
 def GA(colony,  mutation_rate, coded_words, dictionary):
     population = colony
     colony_size = colony.shape[0]
     itter=0
     while True:
        best_acc , selected = selection(population, colony_size, coded_words, dictionary)
        if best_acc == 1 or itter == 35:
            return selected[0,:], best_acc
        all_species = cross_over(selected)
        immortals = mutation(all_species[colony_size:2*colony_size,:], mutation_rate)
        population = np.empty(( all_species.shape[0] + immortals.shape[0] , 26  ), dtype = object)
        population[0: all_species.shape[0] ] = all_species
        population[ all_species.shape[0] : all_species.shape[0] + immortals.shape[0] ] = immortals
        itter += 1

In [17]:
def cal_freq_match(dictionary, coded_words):
    all_freq = {}
    for word in dictionary:
        for i in str(word[0]): 
            if i in all_freq: 
                all_freq[i] += 1
            else: 
                all_freq[i] = 1
    coded_word_freq = {}
    for word in coded_words:
        for i in str(word[0]): 
            if i in coded_word_freq: 
                coded_word_freq[i] += 1
            else: 
                coded_word_freq[i] = 1
    dictionary_sort = sorted(all_freq.items(), key=lambda x: x[1], reverse=True)
    coded_words_sort = sorted(coded_word_freq.items(), key=lambda x: x[1], reverse=True)
    matched = {}
    for i in range(26):
        matched[ dictionary_sort[i][0] ] = coded_words_sort[i][0]
    matched = dict(collections.OrderedDict(sorted(matched.items())))
    return matched


In [18]:
key = []
a = cal_freq_match(dictionary, coded_words)
for letter in a.values():
    key.append(letter)

In [40]:
best_key, accuracy = GA(colony, 0.1, coded_words, dictionary)
best_key

'zyvkosbdixpghtwcarmjlfunqe'

In [41]:
key= ''.join(key)
best_acc_reached = rank(best_key,coded_words,dictionary)*100

In [42]:
print('Best accuracy reached '+ str(best_acc_reached) + '%'+ ' for key : '+ str(best_key))

Best accuracy reached 85.0% for key : zyvkosbdixpghtwcarmjlfunqe
